# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [1]:
import os
import pickle
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import json
import sys

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

In [2]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, project directory is not necessarily the ranch directory

In [3]:
# project directory
root_dir = r"C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin" #os.path.join("D:/merced")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = r"C:\Users\USLP095001\code\NetworkWraner\Ranch" #os.path.join("D:/github/Ranch")

# the folder where SharedStreets extractions live

shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")

# the folder where OSMNX extractions live

osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

In [4]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2024-11-14 16:29:11, INFO: Lasso base directory set as: C:\Users\USLP095001\code\NetworkWraner\Ranch


In [5]:
parameters.standard_crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
for d in [root_dir, external_dir, interim_dir, shst_extract_dir, osm_extract_dir]:
    if os.path.isdir(d):
        print(f"{d} exists!")
        continue
    
    print(f"creating {d}")
    os.makedirs(d)

C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin exists!
C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external exists!
C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\interim exists!
C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\sharedstreets_extract exists!
C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\osmnx_extract exists!


## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [ ]:
# the polygon file for the area

input_polygon_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "merced.shp"
)

In [ ]:
# the SharedStreets extraction file is saved to the 'output_dir' argument

ranch.run_shst_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = shst_extract_dir
)

## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [ ]:
# the OSMNX extraction file is saved to the 'output_dir' argument

ranch.run_osmnx_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = os.path.join(external_dir, "osmnx_extract")
)

## Step 3 - Consolidate SharedStreets and OSMNX extractions

consolidates raw roadway data and create initial roadway networks

In [7]:
# build and returns roaday network object from extraction files

roadway_network = Roadway.create_roadway_network_from_extracts(
    shst_extract_dir = shst_extract_dir,
    osm_extract_dir = osm_extract_dir,
    parameters = parameters,
)

2024-11-14 16:29:20, INFO: Reading sharedstreets data
2024-11-14 16:29:20, INFO: ----------start reading shst extraction data-------------
2024-11-14 16:29:20, INFO: reading shst extraction data : C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\sharedstreets_extract\extract.boundary.0.out.geojson
2024-11-14 16:29:35, INFO: ----------finished reading shst extraction data-------------
2024-11-14 16:29:35, INFO: Removing duplicates in shst extraction data
2024-11-14 16:29:35, INFO: ...before removing duplicates, shst extraction has 99961 geometries.
2024-11-14 16:29:35, INFO: ...after removing duplicates, shst extraction has 99961 geometries.
2024-11-14 16:29:35, INFO: Reading osmnx data
2024-11-14 16:31:30, INFO: Extracting corresponding osm ways for every shst geometry
2024-11-14 16:32:06, INFO: shst extraction has 99961 geometries
2024-11-14 16:32:06, INFO: shst extraction has 107067 osm links
2024-11-14 16:32:06, INFO: ---joining osm shst with osmnx dat

c:\users\uslp095001\code\networkwraner\ranch\ranch\utils.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[x].fillna("", inplace=True)
c:\users\uslp095001\code\networkwraner\ranch\ranch\utils.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

2024-11-14 16:32:07, INFO: Start aggregating osm segments to one shst link for forward links
2024-11-14 16:32:32, INFO: Start aggregating osm segments to one shst link for backward links
2024-11-14 16:32:55, INFO: all
2024-11-14 16:32:58, INFO: Start creating shst nodes
2024-11-14 16:33:03, INFO: Lasso base directory set as: C:\Users\USLP095001\code\NetworkWraner\Ranch


In [ ]:
roadway_network.links_df["data_source"].notna().sum()

In [ ]:
RanchLogger.info("Initial network has {} links".format(roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(roadway_network.shapes_df.shape[0]))

### Optional: If the user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out. These files can also be written out after step 5.  After step 5, roadway network is trimmed to be within county boundary

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step3_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

### Optional: it's more efficient if the user writes out the intermediate network object in pickle

In [ ]:
working_network_filename = os.path.join(interim_dir,"step3_network.pickle")
pickle.dump(roadway_network, open(working_network_filename, 'wb'))

## Step 4 - Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

In [8]:
# define the input 3rd-party file, either shapefile or geojson

input_network_file = os.path.join(external_dir, "sjmodel", "Network", "2015","TCM_MASTER_2A22_102717.shp")

In [9]:
# if the CRS is undefined in the input network file, user needs to specify:

input_crs = CRS("ESRI:102643")

In [26]:
# usually pandas can keep the crs, with the 
input_network_to_conflate = gpd.read_file(input_network_file, crs=input_crs)#, engine="pyogrio")
input_network_to_conflate = input_network_to_conflate.set_crs(input_crs)

In [27]:
print(roadway_network.links_df.crs)

epsg:4326


In [30]:
from ranch import conflate_linestrings

conflate_linestrings.conflate_line_segments(
    roadway_network.links_df.to_crs(input_crs),
    input_network_to_conflate, 
    30,
)


: 

In [17]:
roadway_network.links_df["geometry"].to_file(r"C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\sjmodel\shst_match\ss\test2.gpkg")

In [13]:
# the output is saved to the 'output_dir' argument
# user can specify the match option (reference here) 
# by 'custom_match_option'
# if not specified, use the default
# match option would vary by the input network

ranch.run_shst_match(
    input_network_file = input_network_file,
    input_crs = input_crs,
    output_dir = os.path.join(external_dir, "sjmodel", "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

2024-11-14 16:51:03, INFO: Input network for shst match does not have unique IDs, generating unique IDs
2024-11-14 16:51:03, INFO: Generated 49910 unique IDs for 49910 links in the input network
2024-11-14 16:51:03, INFO: Exporting shst match input - ID-ed geometry file C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.geojson
2024-11-14 16:51:05, INFO: Exporting ID-ed network file C:\Users\USLP095001\code\NetworkWraner\ranch_resources\sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.full.geojson


AssertionError: 

## Step 5 - Tidy Up Roadway

Trim network to be within county boundary. Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name.

In [ ]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "merced.shp"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [ ]:
roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name,
    create_node_link_id = False
)

In [ ]:
# save out the step 5 output as pickle file

working_network_filename = os.path.join(interim_dir,"step5_network.pickle")
pickle.dump(roadway_network, open(working_network_filename, 'wb'))

## Step 5.1 (Optional) - Combine County Roadway Networks Before GTFS Routing

If the user already has the entire roadway network in one roadway object, then skip this step.

This step combines multiple roadway network objects into one, so that a complete roadway network can be used in step 6 - transit routing. This addresses issues like buses crossing multiple counties.

In [ ]:
# inputs are individual county network objects saved out from step 5

merced_roadway_network_pickle_file_name = os.path.join("D:\merced\data\interim\step5_network.pickle")
merced_roadway_network = pickle.load(open(merced_roadway_network_pickle_file_name, 'rb'))

stanislaus_roadway_network_pickle_file_name = os.path.join("D:\stanislaus\data\interim\step5_network.pickle")
stanislaus_roadway_network = pickle.load(open(stanislaus_roadway_network_pickle_file_name, 'rb'))

sanjoaquin_roadway_network_pickle_file_name = os.path.join("D:\sanjoaquin\data\interim\step5_network.pickle")
sanjoaquin_roadway_network = pickle.load(open(sanjoaquin_roadway_network_pickle_file_name, 'rb'))

In [ ]:
roadway_network_list = [merced_roadway_network, stanislaus_roadway_network, sanjoaquin_roadway_network]

In [ ]:
# process

roadway_network = ranch.roadway.combine_roadway_network_from_objects(
    roadway_network_list = roadway_network_list,
    parameters = parameters
)

## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [ ]:
# the roadway_network used for step 6 should be the entire region's network

all_county_network_filename = os.path.join(
    'D:/WSP O365/BART-TDLU - Documents/Networks', '21Counties', 'interim',"step5_network.pickle"
)

roadway_network = pickle.load(open(all_county_network_filename, 'rb'))

In [ ]:
# the directory where all gtfs feeds are located
# e.g. gtfs_dir/sjrtd_2015_0127/routes.txt
# e.g. gtfs_dir/bart_2015_0127/routes.txt

gtfs_dir = os.path.join(external_dir, "gtfs", "2021")

In [ ]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    gtfs_dir = gtfs_dir,
    roadway_network= roadway_network,
    parameters=parameters
)

In [ ]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network(
    multithread_shst_match = True,
    multithread_shortest_path = False
)

In [ ]:
# check if shortest path transit builder failed any transit shapes

transit_network.shortest_path_failed_shape_list

In [ ]:
# writes out standard transit network and rail links/nodes if present

transit_network.write_standard_transit(
    path = interim_dir
)

In [ ]:
# write out bus route for review
bus_trip_link_df = gpd.GeoDataFrame(
    transit_network.bus_trip_link_df, 
    crs = roadway_network.links_df.crs)

bus_trip_link_df.drop('wayId', axis = 1).to_file(
    os.path.join(interim_dir, 'bus_routing.geojson'),
    driver = 'GeoJSON'
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ, MAZ

In [ ]:
taz_polygon_file = os.path.join(
    external_dir,
    "taz",
    "SJ_TAZ_Aug2011.shp"
)

In [ ]:
roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = taz_polygon_file
)

## Step 8 - Write out Standard Format

In [ ]:
roadway_network.standard_format(
    county_boundary_file = os.path.join(external_dir,
        "sharedstreets_extract", 
        "merced.shp"),
    county_variable_name = 'NAME'
)

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step8_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)